<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Ananalytics/blob/main/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print('Hello')

Hello


In [ ]:
print('Hello Sarvesh GlobalPandey')